Big thanks to the notebooks https://www.kaggle.com/code/leehomhuang/lgb-baseline-train, https://www.kaggle.com/code/meli19/lgb-kf-baseline, https://www.kaggle.com/code/dangnguyen97/lgb-kf-with-optuna/notebook and https://www.kaggle.com/code/ravi20076/optiver-baseline-models. If you like this notebook please go and give them thumbs up too.

![](https://www.factinate.com/wp-content/uploads/2018/04/hoarding-facts-factinate-new.png)

In [1]:
%%time 

# General library imports:-
from IPython.display import display_html, clear_output, Markdown;
from gc import collect;
import copy
from copy import deepcopy;
import pandas as pd;
import numpy as np;
import joblib;
from os import system, getpid, walk;
from psutil import Process;
import ctypes;
libc = ctypes.CDLL("libc.so.6");

from pprint import pprint;
from colorama import Fore, Style, init;
from warnings import filterwarnings;
filterwarnings('ignore');

from tqdm.notebook import tqdm;

import lightgbm as lgb
import gc
from itertools import combinations
import warnings
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from warnings import simplefilter
import joblib

warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

print();
collect();


CPU times: user 1.44 s, sys: 564 ms, total: 2 s
Wall time: 2.77 s


In [2]:
%%time 

# Model development:-
from sklearn.model_selection import (RepeatedStratifiedKFold as RSKF, 
                                     StratifiedKFold as SKF,
                                     KFold, 
                                     RepeatedKFold as RKF, 
                                     cross_val_score);

from lightgbm import log_evaluation, early_stopping, LGBMRegressor as LGBMR;
from xgboost import XGBRegressor as XGBR;
from catboost import CatBoostRegressor as CBR;
from sklearn.ensemble import HistGradientBoostingRegressor as HGBR;
from sklearn.metrics import mean_absolute_error as mae, make_scorer;

print();
collect();


CPU times: user 286 ms, sys: 87.2 ms, total: 373 ms
Wall time: 918 ms


In [3]:
%%time

# Defining global configurations and functions:-

# Color printing    
def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    "Prints color outputs using colorama using a text F-string";
    print(style + color + text + Style.RESET_ALL); 
    
def GetMemUsage():
    """
    This function defines the memory usage across the kernel. 
    Source-
    https://stackoverflow.com/questions/61366458/how-to-find-memory-usage-of-kaggle-notebook
    """;
    
    pid = getpid();
    py = Process(pid);
    memory_use = py.memory_info()[0] / 2. ** 30;
    return f"RAM memory GB usage = {memory_use :.4}";

# Making sklearn pipeline outputs as dataframe:-
from sklearn import set_config; 
set_config(transform_output = "pandas");
pd.set_option('display.max_columns', 50);
pd.set_option('display.max_rows', 50);

print();
collect();


CPU times: user 94 ms, sys: 441 µs, total: 94.5 ms
Wall time: 93 ms


In [4]:
%%time 

# Configuration class:-
class CFG:
    """
    Configuration class for parameters and CV strategy for tuning and training
    Please use caps lock capital letters while filling in parameters
    """;
    
    # Data preparation:-   
    version_nb         = 5;
    test_req           = "N";
    test_frac          = 0.01;
    load_tr_data       = "N";
    gpu_switch         = "OFF"; 
    state              = 42;
    target             = 'target';
    
    path               = f"/kaggle/input/optiver-memoryreduceddatasets/";
    test_path          = f"/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv";
    df_choice          = f"XTrIntCmpNewFtre.parquet";
    mdl_path           = f'/kaggle/working/BaselineML/';
    inf_path           = f'/kaggle/input/optiverbaselinemodels/';
     
    # Model Training:-
    methods            = ["LGBMR", "CBR", "HGBR"];
    ML                 = "N";
    n_splits           = 5;
    n_repeats          = 1;
    nbrnd_erly_stp     = 100 ;
    mdlcv_mthd         = 'KF';
    
    # Ensemble:-    
    ensemble_req       = "N";
    enscv_mthd         = "KF";
    metric_obj         = 'minimize';
    ntrials            = 10 if test_req == "Y" else 200;
    ens_weights        = [0.54, 0.44, 0.02];
    
    # Inference:-
    inference_req      = "Y";
    
    # Global variables for plotting:-
    grid_specs = {'visible': True, 'which': 'both', 'linestyle': '--', 
                  'color': 'lightgrey', 'linewidth': 0.75
                 };
    title_specs = {'fontsize': 9, 'fontweight': 'bold', 'color': 'tab:blue'};

print();
PrintColor(f"--> Configuration done!\n");
collect();

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED);


--> Configuration done!


RAM memory GB usage = 0.2535
CPU times: user 93.7 ms, sys: 0 ns, total: 93.7 ms
Wall time: 93.5 ms


In [5]:
%%time 

# Commonly used CV strategies for later usage:-
all_cv= {'KF'  : KFold(n_splits= CFG.n_splits, shuffle = True, random_state= CFG.state),
         'RKF' : RKF(n_splits= CFG.n_splits, n_repeats = CFG.n_repeats, random_state= CFG.state),
         'RSKF': RSKF(n_splits= CFG.n_splits, n_repeats = CFG.n_repeats, random_state= CFG.state),
         'SKF' : SKF(n_splits= CFG.n_splits, shuffle = True, random_state= CFG.state)
        };

# Defining the competition metric:-
def ScoreMetric(ytrue, ypred)-> float:
    """
    This function calculates the metric for the competition. 
    ytrue- ground truth array
    ypred- predictions
    returns - metric value (float)
    """;
    
    return mae(ytrue, ypred);

# Designing a custom scorer to use in cross_val_predict and cross_val_score:-
myscorer = make_scorer(ScoreMetric, greater_is_better = False, needs_proba=False,);

print();
collect();

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED);



RAM memory GB usage = 0.2535
CPU times: user 105 ms, sys: 0 ns, total: 105 ms
Wall time: 104 ms


In [6]:
%%time

def goto_conversion(listOfOdds, total = 1, eps = 1e-6, isAmericanOdds = False):
    "Source - https://www.kaggle.com/code/kaito510/goto-conversion-optiver-baseline-models";

    #Convert American Odds to Decimal Odds
    if isAmericanOdds:
        for i in range(len(listOfOdds)):
            currOdds = listOfOdds[i];
            isNegativeAmericanOdds = currOdds < 0;
            if isNegativeAmericanOdds:
                currDecimalOdds = 1 + (100/(currOdds*-1));
            else: 
                #Is non-negative American Odds
                currDecimalOdds = 1 + (currOdds/100);
            listOfOdds[i] = currDecimalOdds;

    #Error Catchers
    if len(listOfOdds) < 2:
        raise ValueError('len(listOfOdds) must be >= 2');
    if any(x < 1 for x in listOfOdds):
        raise ValueError('All odds must be >= 1, set isAmericanOdds parameter to True if using American Odds');

    #Computation:-
    #initialize probabilities using inverse odds
    listOfProbabilities = [1/x for x in listOfOdds];
    
    #compute the standard error (SE) for each probability
    listOfSe = [pow((x-x**2)/x,0.5) for x in listOfProbabilities];
    
    #compute how many steps of SE the probabilities should step back by
    step = (sum(listOfProbabilities) - total)/sum(listOfSe) ;
    outputListOfProbabilities = [min(max(x - (y*step),eps),1) for x,y in zip(listOfProbabilities, listOfSe)];
    return outputListOfProbabilities;

def zero_sum(listOfPrices, listOfVolumes):
    """
    Source - https://www.kaggle.com/code/kaito510/goto-conversion-optiver-baseline-models
    """;
    
    #compute standard errors assuming standard deviation is same for all stocks
    listOfSe = [x**0.5 for x in listOfVolumes];
    step = sum(listOfPrices)/sum(listOfSe);
    outputListOfPrices = [x - (y*step) for x,y in zip(listOfPrices, listOfSe)];
    return outputListOfPrices;

collect();

CPU times: user 98 ms, sys: 489 µs, total: 98.4 ms
Wall time: 98.5 ms


In [7]:
%%time 

if (CFG.load_tr_data == "Y" or CFG.ML == "Y") and CFG.test_req == "Y":
    if isinstance(CFG.test_frac, float):
        X = pd.read_parquet(CFG.path + CFG.df_choice).sample(frac = CFG.test_frac);
    else:
        X = pd.read_parquet(CFG.path + CFG.df_choice).sample(n = CFG.test_frac);
        
    y = pd.read_parquet(CFG.path + f"Ytrain.parquet").loc[X.index].squeeze();
    PrintColor(f"---> Sampled train shapes for code testing = {X.shape} {y.shape}", 
               color = Fore.RED);
    X.index, y.index = range(len(X)), range(len(y));
    
    PrintColor(f"\n---> Train set columns for model development");
    pprint(X.columns, width = 100, depth = 1, indent = 5);
    print();

elif CFG.load_tr_data == "Y" or CFG.ML == "Y":
    X = pd.read_parquet(CFG.path + CFG.df_choice);
    y = pd.read_parquet(CFG.path + f"Ytrain.parquet").squeeze();  
    PrintColor(f"---> Train shapes for code testing = {X.shape} {y.shape}");

elif CFG.load_tr_data != "Y" or CFG.inference_req == "Y":
    PrintColor(f"---> Train data is not required as we are infering from the model");
    
print();
collect();
libc.malloc_trim(0);

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED);

---> Train data is not required as we are infering from the model


RAM memory GB usage = 0.2521
CPU times: user 98.3 ms, sys: 0 ns, total: 98.3 ms
Wall time: 98.8 ms


In [8]:
%%time 

# Initializing model I-O:-

if CFG.ML == "Y":
    Mdl_Master = \
    {'CBR': CBR(**{'task_type'           : "GPU" if CFG.gpu_switch == "ON" else "CPU",
                   'objective'           : "MAE",
                   'eval_metric'         : "MAE",
                   'bagging_temperature' : 0.5,
                   'colsample_bylevel'   : 0.7,
                   'iterations'          : 500,
                   'learning_rate'       : 0.065,
                   'od_wait'             : 25,
                   'max_depth'           : 7,
                   'l2_leaf_reg'         : 1.5,
                   'min_data_in_leaf'    : 1000,
                   'random_strength'     : 0.65, 
                   'verbose'             : 0,
                   'use_best_model'      : True,
                  }
               ), 

      'LGBMR': LGBMR(**{'device'            : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                        'objective'         : 'regression_l1',
                        'boosting_type'     : 'gbdt',
                        'random_state'      : CFG.state,
                        'colsample_bytree'  : 0.7,
                        'subsample'         : 0.65,
                        'learning_rate'     : 0.065,
                        'max_depth'         : 6,
                        'n_estimators'      : 500,
                        'num_leaves'        : 150,  
                        'reg_alpha'         : 0.01,
                        'reg_lambda'        : 3.25,
                        'verbose'           : -1,
                       }
                    ),

      'XGBR': XGBR(**{'tree_method'        : "gpu_hist" if CFG.gpu_switch == "ON" else "hist",
                      'objective'          : 'reg:absoluteerror',
                      'random_state'       : CFG.state,
                      'colsample_bytree'   : 0.7,
                      'learning_rate'      : 0.07,
                      'max_depth'          : 6,
                      'n_estimators'       : 500,                         
                      'reg_alpha'          : 0.025,
                      'reg_lambda'         : 1.75,
                      'min_child_weight'   : 1000,
                      'early_stopping_rounds' : CFG.nbrnd_erly_stp,
                     }
                  ),

      "HGBR" : HGBR(loss              = 'squared_error',
                    learning_rate     = 0.075,
                    early_stopping    = True,
                    max_iter          = 200,
                    max_depth         = 6,
                    min_samples_leaf  = 1500,
                    l2_regularization = 1.75,
                    scoring           = myscorer,
                    random_state      = CFG.state,
                   )
    };

print();
collect();

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED);




RAM memory GB usage = 0.2523
CPU times: user 94.7 ms, sys: 515 µs, total: 95.2 ms
Wall time: 95 ms


In [9]:
%%time 

if CFG.ML == "Y":
    # Initializing the models from configuration class:-
    methods = CFG.methods;

    # Initializing a folder to store the trained and fitted models:-
    system('mkdir BaselineML');

    # Initializing the model path for storage:-
    model_path = CFG.mdl_path;

    # Initializing the cv object:-
    cv = all_cv[CFG.mdlcv_mthd];
        
    # Initializing score dataframe:-
    Scores = pd.DataFrame(index = range(CFG.n_splits * CFG.n_repeats),
                          columns = methods).fillna(0).astype(np.float32);
    
    FtreImp = pd.DataFrame(index = X.columns, columns = [methods]).fillna(0);

print();
collect();
libc.malloc_trim(0);

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED);



RAM memory GB usage = 0.2521
CPU times: user 97.5 ms, sys: 1.41 ms, total: 98.9 ms
Wall time: 98 ms


In [10]:
%%time 

if CFG.ML == "Y":
    PrintColor(f"\n{'=' * 25} ML Training {'=' * 25}\n");
    
    # Initializing CV splitting:-       
    for fold_nb, (train_idx, dev_idx) in tqdm(enumerate(cv.split(X, y)), 
                                              f"{CFG.mdlcv_mthd} CV {CFG.n_splits}x{CFG.n_repeats}"
                                             ): 
        # Creating the cv folds:-    
        Xtr  = X.iloc[train_idx];   
        Xdev = X.iloc[dev_idx];
        ytr  = y.iloc[train_idx];
        ydev = y.iloc[dev_idx];
        
        PrintColor(f"-------> Fold{fold_nb} <-------");
        # Fitting the models:- 
        for method in methods:
            model = Mdl_Master[method];
            if method == "LGBMR":
                model.fit(Xtr, ytr, 
                          eval_set = [(Xdev, ydev)], 
                          verbose = 0, 
                          eval_metric = "mae",
                          callbacks = [log_evaluation(0,), 
                                       early_stopping(CFG.nbrnd_erly_stp, verbose = False)], 
                         );

            elif method == "XGBR":
                model.fit(Xtr, ytr, 
                          eval_set = [(Xdev, ydev)], 
                          verbose = 0, 
                          eval_metric = "mae",
                         );  

            elif method == "CBR":
                model.fit(Xtr, ytr, 
                          eval_set = [(Xdev, ydev)], 
                          verbose = 0, 
                          early_stopping_rounds = CFG.nbrnd_erly_stp,
                         ); 

            else:
                model.fit(Xtr, ytr);

            #  Saving the model for later usage:-
            joblib.dump(model, CFG.mdl_path + f'{method}V{CFG.version_nb}Fold{fold_nb}.model');
            
            # Creating OOF scores:-
            score = ScoreMetric(ydev, model.predict(Xdev));
            Scores.at[fold_nb, method] = score;
            num_space = 6- len(method);
            PrintColor(f"---> {method} {' '* num_space} OOF = {score:.5f}", 
                       color = Fore.MAGENTA);  
            del num_space, score;
            
            # Collecting feature importances:-
            try:
                FtreImp[method] = \
                FtreImp[method].values + (model.feature_importances_ / (CFG.n_splits * CFG.n_repeats));
            except:
                pass;
            
            collect();
            
        PrintColor(GetMemUsage());
        print();
        del Xtr, ytr, Xdev, ydev;
        collect();
    
    clear_output();
    PrintColor(f"\n---> OOF scores across methods <---\n");
    Scores.index.name = "FoldNb";
    Scores.index = Scores.index + 1;
    display(Scores.style.format(precision = 5).\
            background_gradient(cmap = "Pastel1")
           );
    
    PrintColor(f"\n---> Mean OOF scores across methods <---\n");
    display(Scores.mean());
    
    try: FtreImp.to_csv(CFG.mdl_path + f"FtreImp_V{CFG.version_nb}.csv");
    except: pass;
        
collect();
print();
libc.malloc_trim(0);

PrintColor(f"\n" + GetMemUsage(), color = Fore.GREEN);



RAM memory GB usage = 0.2521
CPU times: user 98.4 ms, sys: 259 µs, total: 98.7 ms
Wall time: 97.7 ms


In [11]:
%%time 

def MakeFtre(df : pd.DataFrame, prices: list) -> pd.DataFrame:
    """
    This function creates new features using the price columns. This was used in a baseline notebook as below-
    https://www.kaggle.com/code/yuanzhezhou/baseline-lgb-xgb-and-catboost
    
    Inputs-
    df:- pd.DataFrame -- input dataframe
    cols:- price columns for transformation
    
    Returns-
    df:- pd.DataFrame -- dataframe with extra columns
    """;
    
    features = ['overall_medvol', "first5min_medvol", "last5min_medvol",
                'seconds_in_bucket', 'imbalance_buy_sell_flag',
                'imbalance_size', 'matched_size', 'bid_size', 'ask_size',
                'reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap',
                'imb_s1', 'imb_s2'
               ];
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)').astype(np.float32);
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)').astype(np.float32);
       
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                df[f'{a}_{b}_imb'] = df.eval(f'({a}-{b})/({a}+{b})');
                features.append(f'{a}_{b}_imb'); 
                    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = df[[a,b,c]].max(axis=1);
                    min_ = df[[a,b,c]].min(axis=1);
                    mid_ = df[[a,b,c]].sum(axis=1)-min_-max_;

                    df[f'{a}_{b}_{c}_imb2'] = ((max_-mid_)/(mid_-min_)).astype(np.float32);
                    features.append(f'{a}_{b}_{c}_imb2');
    
    return df[features];

print();
collect();


CPU times: user 94.7 ms, sys: 0 ns, total: 94.7 ms
Wall time: 93.8 ms


In [12]:
%%time 

# Creating the testing environment:-
if CFG.inference_req == "Y":
    try: 
        del X, y;
    except: 
        pass;
        
    prices = ['reference_price', 'far_price', 'near_price', 'bid_price', 'ask_price', 'wap'];
    
    # Making the test environment for inferencing:-
    import optiver2023;
    try: 
        env = optiver2023.make_env();
        iter_test = env.iter_test();
        PrintColor(f"\n---> Curating the inference environment");
    except: 
        pass;
    
    # Collating a list of models to be used for inferencing:-
    models = [];

    # Loading the models for inferencing:-
    if CFG.ML != "Y": 
        model_path = CFG.inf_path;
        PrintColor(f"---> Loading models from the input data for the kernel - V{CFG.version_nb}\n", 
                  color = Fore.RED);
    elif CFG.ML == "Y": 
        model_path = CFG.mdl_path;
        PrintColor(f"---> Loading models from the working directory for the kernel\n");
    
    # Loading the models from the models dataframe:-
    mdl_lbl = [];
    for _, _, filename in walk(model_path):
        mdl_lbl.extend(filename);

    models = [];
    for filename in mdl_lbl:
        models.append(joblib.load(model_path + f"{filename}"));
        
    mdl_lbl    = [m.replace(r".model", "") for m in mdl_lbl];
    model_dict = {l:m for l,m in zip(mdl_lbl, models)};
    PrintColor(f"\n---> Trained models\n");    
    pprint(np.array(mdl_lbl), width = 100, indent = 10, depth = 1);  
       
print();
collect();  
libc.malloc_trim(0);
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED); 


---> Curating the inference environment
---> Loading models from the input data for the kernel - V5


---> Trained models

array(['CBRV2Fold2', 'CBRV3Fold5', 'LGBMRV3Fold4', 'CBRV5Fold3',
       'LGBMRV2Fold3', 'HGBRFold0', 'HGBRFold1', 'CBRV3Fold8',
       'LGBMRV3Fold1', 'CBRV5Fold1', 'CBRV3Fold6', 'LGBMRFold3',
       'CBRV4Fold1', 'LGBMRV5Fold0', 'HGBRFold4', 'HGBRV5Fold4',
       'LGBMRFold1', 'CBRV4Fold3', 'LGBMRV3Fold3', 'LGBMRV3Fold0',
       'CBRV3Fold4', 'CBRV2Fold0', 'XGBRFold2', 'LGBMRV3Fold7',
       'HGBRFold3', 'CBRV3Fold13', 'CBRV4Fold2', 'CBRV2Fold3',
       'CBRV3Fold1', 'LGBMRFold4', 'LGBMRV5Fold4', 'LGBMRV4Fold0',
       'CBRV2Fold1', 'LGBMRV3Fold8', 'CBRV3Fold7', 'LGBMRV3Fold2',
       'LGBMRV5Fold3', 'CBRV4Fold4', 'CBRV3Fold10', 'HGBRV5Fold1',
       'CBRV5Fold4', 'CBRFold0', 'LGBMRV3Fold10', 'LGBMRV2Fold4',
       'LGBMRV3Fold11', 'CBRFold1', 'LGBMRFold2', 'LGBMRV2Fold0',
       'LGBMRV3Fold12', 'LGBMRV4Fold2', 'LGBMRV5Fold1', 'CBRV3Fold12',
       'CBRFold3', '

In [13]:
is_train = False
is_infer = True
N_Folds = 4

train = pd.read_csv('../input/optiver-trading-at-the-close/train.csv')
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].std() + train.groupby('stock_id')['ask_size'].std()
train = train.dropna(subset=['target'])

def feat_eng(df):
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    df['bid_ask_volume_diff'] = df['ask_size'] - df['bid_size']
    df['mid_price'] = (df['ask_price'] + df['bid_price']) / 2
    df['bid_plus_ask_sizes'] = df['bid_size'] + df['ask_size']
    df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
    df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0)
        
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(prices, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    
        
    df.drop(columns=[
        'date_id', 
    ], inplace=True)
        
    gc.collect()
    
    return df

y = train['target'].values
X = feat_eng(train.drop(columns='target'))

y_min = np.min(y)
y_max = np.max(y)

params = {
    'learning_rate': 0.018,
    'max_depth': 9,
    'n_estimators': 600,
    'num_leaves': 440,
    'objective': 'mae',
    'random_state': 42,
    'reg_alpha': 0.01,
    'reg_lambda': 0.01
}


kf = KFold(n_splits=N_Folds, shuffle=True, random_state=42)
mae_scores = []


def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step
    
    return out


if is_infer:
    predictions = []


In [14]:
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].std() + train.groupby('stock_id')['ask_size'].std()
max_sizes = train.groupby('stock_id')['bid_size'].max() + train.groupby('stock_id')['ask_size'].max()
min_sizes = train.groupby('stock_id')['bid_size'].min() + train.groupby('stock_id')['ask_size'].min()
mean_sizes = train.groupby('stock_id')['bid_size'].mean() + train.groupby('stock_id')['ask_size'].mean()
first_sizes = train.groupby('stock_id')['bid_size'].first() + train.groupby('stock_id')['ask_size'].first()
last_sizes = train.groupby('stock_id')['bid_size'].last() + train.groupby('stock_id')['ask_size'].last()

def feature_eng(df):
    cols = [c for c in df.columns if c not in ['row_id', 'date_id','time_id']]
    df = df[cols]
    
    #匹配失败数量和匹配成功数量的比率
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    #供需市场的差额
    df['bid_ask_volume_diff'] = df['ask_size'] - df['bid_size']
    #供需市场总和
    df['bid_plus_ask_sizes'] = df['bid_size'] + df['ask_size']
    
    #供需价格的均值
    df['mid_price'] = (df['ask_price'] + df['bid_price']) / 2
    
    #整体数据情况
    df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
    df['max_size'] = df['stock_id'].map(max_sizes.to_dict())
    df['min_size'] = df['stock_id'].map(min_sizes.to_dict())
    df['mean_size'] = df['stock_id'].map(mean_sizes.to_dict())
    df['first_size'] = df['stock_id'].map(first_sizes.to_dict())    
    df['last_size'] = df['stock_id'].map(last_sizes.to_dict())       
    
    #整体市场规模和当前的市场规模比较
    df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0)
    
    prices = ['reference_price', 'far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    #价格之间做差，做差/求和
    for c in combinations(prices, 2):
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]} - {c[1]})/({c[0]} + {c[1]})')
        
    for c in combinations(prices, 3):
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1) - min_ - max_
        
        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    
    #另外的特征
    #df['less_5min'] = df['seconds_in_bucket'].apply(lambda x: 1 if x < 300 else 0)
    #df['5min_8min'] = df['seconds_in_bucket'].apply(lambda x: 1 if 300 <= x else 0)
    #df['more_8min'] = df['seconds_in_bucket'].apply(lambda x: 1 if 480 <= x else 0)
        
    df_encoded = pd.get_dummies(df['imbalance_buy_sell_flag'])
    df_encoded = df_encoded.rename(columns={-1: 'sell-side imbalance', 0: 'no imbalance', 1: 'buy-side imbalance'})

    df = pd.concat([df, df_encoded], axis=1)
    
    
    gc.collect()
    
    return df

In [15]:
%%time 

if CFG.inference_req == "Y":
    print();
    counter = 0;
    
    try:
        median_vol = pd.read_csv(CFG.path + f"MedianVolV2.csv", index_col = ['Unnamed: 0']);
    except:
        median_vol = pd.read_csv(CFG.path + f"MedianVolV2.csv"); 
    median_vol.index.name = "stock_id";
    median_vol = median_vol[['overall_medvol', "first5min_medvol", "last5min_medvol"]];
    
    for test, revealed_targets, sample_prediction in iter_test:
        if counter >= 99: num_space = 1;
        elif counter >= 9: num_space = 2;
        else: num_space = 3;
        
        PrintColor(f"{counter + 1}. {' ' * num_space} Inference", color = Fore.MAGENTA);
        testforsecond = copy.deepcopy(test)
        test  = test.merge(median_vol, how = "left", left_on = "stock_id", right_index = True);
        Xtest = MakeFtre(test, prices = prices);
        del num_space;
        
        # Curating model predictions across methods and folds:-        
        preds = pd.DataFrame(columns = CFG.methods, index = Xtest.index).fillna(0);
        for method in CFG.methods:
            for mdl_lbl, mdl in model_dict.items():
                if mdl_lbl.startswith(f"{method}V{CFG.version_nb}"):
                    if CFG.test_req == "Y":
                        print(mdl_lbl);
                    else:
                        pass;
                    preds[method] = preds[method] + mdl.predict(Xtest)/ (CFG.n_splits * CFG.n_repeats);
        
        # Curating the weighted average model predictions:-       
        sample_prediction['target'] = \
        np.average(preds.values, weights= CFG.ens_weights, axis=1);
        
        # Source - https://www.kaggle.com/code/kaito510/goto-conversion-optiver-baseline-models     
        sample_prediction['target'] = \
        zero_sum(sample_prediction['target'], test.loc[:,'bid_size'] + test.loc[:,'ask_size'])
        
        #35
        feat = feat_eng(testforsecond)
        fold_prediction = 0
        for fold in range(0, N_Folds):
            model_filename = f"/kaggle/input/lgb-models-optv2/model_fold_{fold+1}.pkl"
            m = joblib.load(model_filename)
            fold_prediction += m.predict(feat)  
            
        for fold in range(0, N_Folds):
            model_filename = f"/kaggle/input/lgb-kf-with-optuna/model_fold_{fold+1}.pkl"
            m = joblib.load(model_filename)
            fold_prediction += m.predict(feat)   
        
        fold_prediction /= (2 * N_Folds)
        fold_prediction = zero_sum(fold_prediction, test.loc[:,'bid_size'] + test.loc[:,'ask_size'])
        clipped_predictions = np.clip(fold_prediction, y_min, y_max)
        
        sample_prediction['target'] = 0.55 * clipped_predictions + 0.45 * sample_prediction['target']
        feat = feature_eng(testforsecond)
        fold_prediction = 0
        for fold in range(0, N_Folds):
            model_filename = f"/kaggle/input/lgb-baseline-train/lgb-models-optv2/model_fold_{fold+1}.pkl"
            m = joblib.load(model_filename)
            fold_prediction += m.predict(feat)   
        
        fold_prediction /= N_Folds
        fold_prediction = zero_sum(fold_prediction, test.loc[:,'bid_size'] + test.loc[:,'ask_size'])
        clipped_predictions = np.clip(fold_prediction, y_min, y_max)
        
        sample_prediction['target'] = 0.6 * clipped_predictions + 0.4 * sample_prediction['target']
        
        try: 
            env.predict(sample_prediction);
        except: 
            PrintColor(f"---> Submission did not happen as we have the file already");
            pass;
        
        counter = counter+1;
        collect();
    
    PrintColor(f"\n---> Submission file\n");
    display(sample_prediction.head(10));
            
print();
collect();  
libc.malloc_trim(0);
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED); 


This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
1.     Inference
2.     Inference
3.     Inference
4.     Inference
5.     Inference
6.     Inference
7.     Inference
8.     Inference
9.     Inference
10.    Inference
11.    Inference
12.    Inference
13.    Inference
14.    Inference
15.    Inference
16.    Inference
17.    Inference
18.    Inference
19.    Inference
20.    Inference
21.    Inference
22.    Inference
23.    Inference
24.    Inference
25.    Inference
26.    Inference
27.    Inference
28.    Inference
29.    Inference
30.    Inference
31.    Inference
32.    Inference
33.    Inference
34.    Inference
35.    Inference
36.    Inference
37.    Inference
38.    Inference
39.    Inference
40.    Inference
41.    Inference
42.    Inference
43.    Inference
44.    Inference
45.    Inference
46.    Inference
47.    Inference
48.    Inference
49.    Inference
50.    Inference
51.    Inference
52.    

,row_id,target
0,480_540_0,-1.282597
1,480_540_1,-0.083771
2,480_540_2,0.614632
3,480_540_3,-1.969221
4,480_540_4,-1.008911
5,480_540_5,3.027940
6,480_540_6,1.287200
7,480_540_7,-1.565255
8,480_540_8,0.707464
9,480_540_9,-0.506127




RAM memory GB usage = 3.967
CPU times: user 9min 45s, sys: 6.26 s, total: 9min 51s
Wall time: 4min 43s
